# TRAVAUX PRATIQUES : Flow optical - Suivi de mouvementdans une scene
### Reprise  - module vision par ordinateur

# Préparé Par : Léon Jean-Bertrand Fritzner


#### Enseignante : Dr NGUYEN Thi Oanh

Date: 20/07/2022


### Resultats de la 1ere partie du TP3: Estimation du flot optique dans une séquence d'images

In [ ]:
!pip install opencv-python==4.5.5.62
# !pip install tensorflow

### Importation des bibliothèques requises

In [ ]:
import numpy as np
import cv2 as cv
from IPython.display import Video
import matplotlib.pyplot as plt

# 1. Chargement de la video et Estimation du flot optique dans une séquence d'images

In [ ]:
#chargement de la vidéo source
cap = cv.VideoCapture("/home/birthou13/Desktop/Vision par ordinateur/VISION/ADL-Rundle-6-raw.webm")

ret, frame1 = cap.read()

#frame precedant recuérer en niveau de gris
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
hsv[...,1] = 255

#pour chaque frame, appliqué le calcul de flow
idx_frame=1

while(1):
    ret, frame2 = cap.read()
    if ret==False: #s'il n'y aplus de frame, si la vidéo est a la fin, sortir de la boucle
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    
   
    #on calcul le flow à partir du frame précedant 
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 10, 3, 5, 0.5,0)
    
    #Recupération du vecteur de flow optique dense (l'angele et la magnetitude) avec la fonction cartToPolar()  d'OpenCV
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    
    # Pour avoir les valeurs normalisées en nombre e ntier entre 0 et 255, nous convertissons les pixels de HSV à BGR
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    # récupréation
    
    numpy_horizontal = np.hstack((frame2, bgr))
    # cv.namedWindow("video playin", cv.WINDOW_AUTOSIZE)
    # cv.resizeWindow('image', 100,80) 
    cv.resizeWindow("Image", 100, 80)
  
    cv.imshow('Image et ', numpy_horizontal)
    
    #convertir en de BGR en RGB pour l'a
    frame2=cv.cvtColor(frame2,cv.COLOR_BGR2RGB)
    rgb=cv.cvtColor(bgr,cv.COLOR_BGR2RGB)   
    k = cv.waitKey(30) & 0xff

    if k == 27:
        break
    elif k == ord('s'): #nous pouvons sauvegarder le frame courant dans le reppertoir "images" en apuyant sur la lettre "s"
        cv.imwrite('images/'+ str(idx_frame)+'frame_courant.png',frame2)
        cv.imwrite('images/'+ str(idx_frame)+'flow_optic_correspondant.png',rgb)
    prvs = next

    
    #Visualisation des frames courrant et de son flow optique
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.imshow(frame2)
    plt.title("Frame " + str(idx_frame))
    plt.subplot(1,2,2)
    plt.imshow(rgb)
    plt.title("Flow optique dense "+ str(idx_frame))
    plt.show()
    idx_frame=idx_frame+1
#libération de la mémoire
cap.release()
cv.destroyAllWindows()

# 2. Resultats de la deuxieme partie : Segmentation des objets en mouvement

## Seuillage avec 1 seuil

In [ ]:
#chargement de la vidéo source
cap = cv.VideoCapture("/home/birthou13/Desktop/Vision par ordinateur/VISION/ADL-Rundle-6-raw.webm")

ret, frame1 = cap.read()

#frame precedant recuérer en niveau de gris
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
#pour chaque frame, appliqué le calcul de flow
idx_frame=1
hsv[...,0] = 255 # angle
hsv[...,1] = 255 # magnétude
s1=50

while(1):
    ret, frame2 = cap.read()
    if ret==False: #s'il n'y aplus de frame, si la vidéo est a la fin, sortir de la boucle
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    
   
    #on calcul le flow à partir du frame précedant 
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 10, 3,5, 0.5,0)
    
    #Recupération du vecteur de flow optique dense (l'angle et la magnetitude) avec la fonction cartToPolar()  d'OpenCV
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    
   # hsv[...,0] = ang*180/np.pi/2
    mag = cv.normalize(mag,None,0,255,cv.NORM_MINMAX) # la luminosité
 
    for i in range(mag.shape[0]): #seuil = 125
        for j in range(mag.shape[1]):
            if mag[i][j]<=s1:
                hsv[...,2][i][j]=0
            else:
                hsv[...,2][i][j]=255 
    
    # Pour avoir les valeurs normalisées en nombre e ntier entre 0 et 255, nous convertissons les pixels de HSV à BGR
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    # récupréation
    
    numpy_horizontal = np.hstack((frame2, bgr))
    # cv.namedWindow("video playin", cv.WINDOW_AUTOSIZE)
    cv.namedWindow("output", cv.WINDOW_NORMAL)
    # cv.resizeWindow("Image", 100, 80)
    cv.resize(frame2, (100, 80))  
    cv.imshow('Image et ', numpy_horizontal)
    
    k = cv.waitKey(30) & 0xff

    if k == 27:
        break
    elif k == ord('s'): #nous pouvons sauvegarder le frame courant dans le reppertoir "images" en apuyant sur la lettre "s"
        cv.imwrite('images/'+ str(idx_frame)+'frame_courant.png',frame2)
        cv.imwrite('images/'+ str(idx_frame)+'flow_optic_correspondant.png',bgr)
    prvs = next
    
    if idx_frame%5==0:
        #Visualisation des frames courrant et de son flow optique
        plt.figure(figsize=(10,10))
        plt.subplot(1,2,1)
        plt.imshow(cv.cvtColor(frame2,cv.COLOR_BGR2RGB))
        plt.title("Frame " + str(idx_frame))
        plt.subplot(1,2,2)
        plt.imshow(cv.cvtColor(bgr,cv.COLOR_BGR2RGB))
        plt.title("Flot optic dense "+ str(idx_frame))
        plt.show()
    idx_frame=idx_frame+1
#libération de la mémoire
cap.release()
cv.destroyAllWindows()

## Seuillage avec 2 seuils

In [ ]:
#chargement de la vidéo source
cap = cv.VideoCapture("/home/birthou13/Desktop/Vision par ordinateur/VISION/ADL-Rundle-6-raw.webm")

ret, frame1 = cap.read()

#frame precedant recuérer en niveau de gris
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
#pour chaque frame, appliqué le calcul de flow
idx_frame=1
hsv[...,0] = 255 # angle
hsv[...,1] = 255 # magnétude
s1=30
s2=180

while(1):
    ret, frame2 = cap.read()
    if ret==False: #s'il n'y aplus de frame, si la vidéo est a la fin, sortir de la boucle
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    
   
    #on calcul le flow à partir du frame précedant 
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 10, 3,5, 0.5,0)
    
    #Recupération du vecteur de flow optique dense (l'angle et la magnetitude) avec la fonction cartToPolar()  d'OpenCV
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    
   # hsv[...,0] = ang*180/np.pi/2
    mag = cv.normalize(mag,None,0,255,cv.NORM_MINMAX) # la luminosité
 
    for i in range(mag.shape[0]): #seuil = 125
        for j in range(mag.shape[1]):
            if mag[i][j]<s1:
                hsv[...,2][i][j]=0
            elif mag[i][j]<s2:
                hsv[...,2][i][j]=128 
            else:
                hsv[...,2][i][j]=255 
    
    # Pour avoir les valeurs normalisées en nombre e ntier entre 0 et 255, nous convertissons les pixels de HSV à BGR
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    # récupréation
    
    numpy_horizontal = np.hstack((frame2, bgr))
    cv.resizeWindow('image', 100,80) 
    cv.imshow('Image et ', numpy_horizontal)
    
    k = cv.waitKey(30) & 0xff

    if k == 27:
        break
    elif k == ord('s'): #nous pouvons sauvegarder le frame courant dans le reppertoir "images" en apuyant sur la lettre "s"
        cv.imwrite('images/'+ str(idx_frame)+'frame_courant.png',frame2)
        cv.imwrite('images/'+ str(idx_frame)+'flow_optic_correspondant.png',bgr)
    prvs = next
    if idx_frame%5==0:
        #Visualisation des frames courrant et de son flow optique
        plt.figure(figsize=(10,10))
        plt.subplot(1,2,1)
        plt.imshow(cv.cvtColor(frame2,cv.COLOR_BGR2RGB))
        plt.title("Frame " + str(idx_frame))
        plt.subplot(1,2,2)
        plt.imshow(cv.cvtColor(bgr,cv.COLOR_BGR2RGB))
        plt.title("Flow optique dense "+ str(idx_frame))
        plt.show()
    idx_frame=idx_frame+1
#libération de la mémoire
cap.release()
cv.destroyAllWindows()

## Seuillage avec 3 seuils

In [ ]:
#chargement de la vidéo source
cap = cv.VideoCapture("/home/birthou13/Desktop/Vision par ordinateur/VISION/ADL-Rundle-6-raw.webm")

ret, frame1 = cap.read()

#frame precedant recuérer en niveau de gris
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
#pour chaque frame, appliqué le calcul de flow
idx_frame=1
hsv[...,0] = 255 # angle
hsv[...,1] = 255 # magnétude
s1=64
s2=128
s3=192
while(1):
    ret, frame2 = cap.read()
    if ret==False: #s'il n'y aplus de frame, si la vidéo est a la fin, sortir de la boucle
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    
   
    #on calcul le flow à partir du frame précedant 
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 10, 3,5, 0.5,0)
    
    #Recupération du vecteur de flow optique dense (l'angle et la magnetitude) avec la fonction cartToPolar()  d'OpenCV
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    
   # hsv[...,0] = ang*180/np.pi/2
    mag = cv.normalize(mag,None,0,255,cv.NORM_MINMAX) # la luminosité

    for i in range(mag.shape[0]): #seuil = 125
        for j in range(mag.shape[1]):
            if mag[i][j]<s1:
                hsv[...,2][i][j]=0
            elif mag[i][j]<s2:
                hsv[...,2][i][j]=s1
            elif mag[i][j]<s3:
                hsv[...,2][i][j]=s2 
            else:
                hsv[...,2][i][j]=255 
    
    # Pour avoir les valeurs normalisées en nombre e ntier entre 0 et 255, nous convertissons les pixels de HSV à BGR
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    # récupréation
    
    numpy_horizontal = np.hstack((frame2, bgr))
    cv.resizeWindow('image', 120,100) 
    cv.imshow('Image et ', numpy_horizontal)
    
    k = cv.waitKey(30) & 0xff

    if k == 27:
        break
    elif k == ord('s'): #nous pouvons sauvegarder le frame courant dans le reppertoir "images" en apuyant sur la lettre "s"
        cv.imwrite('images/'+ str(i)+'frame_courant.png',frame2)
        cv.imwrite('images/'+ str(i)+'flow_optic_correspondant.png',bgr)
    prvs = next
    if idx_frame%5==0:
        #Visualisation des frames courrant et de son flow optique
        plt.figure(figsize=(10,10))
        plt.subplot(1,2,1)
        plt.imshow(cv.cvtColor(frame2,cv.COLOR_BGR2RGB))
        plt.title("Frame " + str(idx_frame))
        plt.subplot(1,2,2)
        plt.imshow(cv.cvtColor(bgr,cv.COLOR_BGR2RGB))
        plt.title("Flow optique dense "+ str(idx_frame))
        plt.show()
    idx_frame=idx_frame+1
#libération de la mémoire
cap.release()
cv.destroyAllWindows()

###  Comparaison des 3 seuillages

In [ ]:
#chargement de la vidéo source
cap = cv.VideoCapture("/home/birthou13/Desktop/Vision par ordinateur/VISION/ADL-Rundle-6-raw.webm")

ret, frame1 = cap.read()

#frame precedant recuérer en niveau de gris
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
#pour chaque frame, appliqué le calcul de flow
idx_frame=1
hsv[...,0] = 255 # angle
hsv[...,1] = 255 # magnétude
s1=100
s21=10
s22=120
s31=25
s32=90
s33=150
while(1):
    ret, frame2 = cap.read()
    if ret==False: #s'il n'y aplus de frame, si la vidéo est a la fin, sortir de la boucle
        break
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    
   
    #on calcul le flow à partir du frame précedant 
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 10, 3,5, 0.5,0)
    
    #Recupération du vecteur de flow optique dense (l'angle et la magnetitude) avec la fonction cartToPolar()  d'OpenCV
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    
   # hsv[...,0] = ang*180/np.pi/2
    mag = cv.normalize(mag,None,0,255,cv.NORM_MINMAX) # la luminosité
    hsv_1=hsv.copy()
    hsv_2=hsv.copy()
    hsv_3=hsv.copy()
    
    #avec un seuil
    for i in range(mag.shape[0]): #seuil = 125
        for j in range(mag.shape[1]):
            if mag[i][j]<s1:
                hsv_1[...,2][i][j]=0
            else:
                hsv_1[...,2][i][j]=255 
   

    #avec 2 seuils
    for i in range(mag.shape[0]): #seuil = 125
        for j in range(mag.shape[1]):
            if mag[i][j]<s21:
                hsv_2[...,2][i][j]=0
            elif mag[i][j]<s22:
                hsv_2[...,2][i][j]=s21 
            else:
                hsv_2[...,2][i][j]=255 
    
    
    #avec 3seuillage
    for i in range(mag.shape[0]): #seuil = 125
        for j in range(mag.shape[1]):
            if mag[i][j]<s31:
                hsv_3[...,2][i][j]=0
            elif mag[i][j]<s32:
                hsv_3[...,2][i][j]=s31
            elif mag[i][j]<s33:
                hsv_3[...,2][i][j]=s32 
            else:
                hsv_3[...,2][i][j]=255     
    
    
    bgr_s1 = cv.cvtColor(hsv_1,cv.COLOR_HSV2BGR)
    bgr_s2 = cv.cvtColor(hsv_2,cv.COLOR_HSV2BGR)
    bgr_s3 = cv.cvtColor(hsv_3,cv.COLOR_HSV2BGR)
    # récupréation
    
    numpy_horizontal = np.hstack((frame2, bgr_s1))
    numpy_vertical = np.vstack((bgr_s2, bgr_s3))
    cv.resizeWindow('image', 120,100) 
    cv.imshow('Image et ', (numpy_horizontal))
    
    k = cv.waitKey(30) & 0xff

    if k == 27:
        break
    elif k == ord('s'): #nous pouvons sauvegarder le frame courant dans le reppertoir "images" en apuyant sur la lettre "s"
        cv.imwrite('images/'+ str(idx_frame)+'frame_courant.png',frame2)
        cv.imwrite('images/'+ str(idx_frame)+'flow_optic_correspondant.png',bgr_s1)
    prvs = next
    
    if(idx_frame%10==0):
        #Visualisation des frames courrant et de son flow optique
        plt.figure(figsize=(12,12))
        plt.subplot(1,4,1)
        plt.imshow(cv.cvtColor(frame2,cv.COLOR_BGR2RGB))
        plt.title("Frame " + str(idx_frame))
        plt.subplot(1,4,2)
        plt.imshow(cv.cvtColor(bgr_s1,cv.COLOR_BGR2RGB))
        plt.title("Flot - 1 seuil ("+ str(idx_frame)+")")
        plt.subplot(1,4,3)
        plt.imshow(cv.cvtColor(bgr_s2,cv.COLOR_BGR2RGB))
        plt.title("Flot - 2 seuils ("+ str(idx_frame)+")")
        plt.subplot(1,4,4)
        plt.imshow(cv.cvtColor(bgr_s3,cv.COLOR_BGR2RGB))
        plt.title("Flot - 3 seuils ("+ str(idx_frame)+")")
        plt.show()
    idx_frame=idx_frame+1
#libération de la mémoire
cap.release()
cv.destroyAllWindows()